In [1]:
import numpy as np

In [ ]:
def mandlebrot(c):
    z = 0 
    iterations = 100
    cSpace = np.linspace(-2, 2, 20)
    plotarray = np.array([],[])

    for c in cSpace: 
        for i in iterations:
            y = z**2 + cSpace[c]
            z = y
            plotarray.append([y,c])
        


    return 
